<a href="https://colab.research.google.com/github/cccadet/dermatologist-ai/blob/master/01_Test1_ISIC_2019_DeepStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pré: Conexão com Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Etapa 1: Instalação das dependências

In [2]:
#!pip install tensorflow-gpu==2.1.0
!pip install tensorflow-gpu
!pip install tqdm
!pip install deepstack
!pip install kaggle
!pip install keras-rectified-adam
!pip install tensorflow-addons
#!pip install tfa-nightly

     |████████████████████████████████| 320.4MB 51kB/s 
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=4c4d969a2e980e4fde82ffd2ba3a6afac09ed59692f51ff8f219e7f9ff652e02
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


### Importação das bibliotecas

In [3]:
import zipfile
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import pandas as pd
import sys
import itertools
from keras_radam import RAdam
import shutil

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.0'

### Otimizações tensorflow

In [4]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
tf.config.optimizer.set_jit(True)

### Fazendo o download da base de dados de imagens

In [5]:
%%time
if (os.path.isdir('./ISIC_2019_Test_Input')==False):
  !wget --no-check-certificate \
      https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip \
      -O ./skin_cancer_test.zip

--2020-08-17 16:05:23--  https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.86.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.86.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3823247963 (3.6G) [application/zip]
Saving to: ‘./skin_cancer_test.zip’

./skin_cancer_test. 100%[===================>]   3.56G  40.8MB/s    in 86s     

2020-08-17 16:06:49 (42.6 MB/s) - ‘./skin_cancer_test.zip’ saved [3823247963/3823247963]

CPU times: user 438 ms, sys: 124 ms, total: 562 ms
Wall time: 1min 27s


## Etapa 2: Pré-processamento

### Configurando os caminhos (paths)

In [6]:
test_dir = "./test"

download_path = './'
#derm_ai_path_V1 = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI'
derm_ai_path = '/content/drive/My Drive/Derm AI/ISIC_2019'
#ground_truth_file = os.path.join(derm_ai_path, 'ground_truth.csv')

In [7]:
def dcp(download_path, path, file):
  if (os.path.isdir(path)==False):
    dataset_path = os.path.join(download_path,file)
    zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
    zip_object.extractall("./")
    zip_object.close()

### Descompactando a base de dados de Treinamento

### Descompactando a base de dados de Testes

In [8]:
%%time
dcp(download_path, './ISIC_2019_Test_Input', 'skin_cancer_test.zip')

CPU times: user 19.1 s, sys: 3.7 s, total: 22.8 s
Wall time: 1min 20s


In [9]:
!rm ./skin_cancer_test.zip

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: InceptionResNetV2 (que estamos usando) suporta: (299x299).

In [10]:
data_gen_test = ImageDataGenerator(rescale=1/255., 
                                   fill_mode="constant"
)

In [11]:
!mkdir ./test
!mkdir ./test/test
!mv ./ISIC_2019_Test_Input/* ./test/test

In [12]:
test_generator  = data_gen_test.flow_from_directory(test_dir, target_size=(299,299), batch_size=32, shuffle=False, class_mode=None)

Found 8238 images belonging to 1 classes.


#### Exibição de 3 imagens carregadas

In [13]:
#x,y = test_generator.next()
#for i in range(0,3):
#    image = x[i]
#    label = y[i]
#    print (label)
#    plt.imshow(image)
#    plt.show()

## Etapa 3: Construindo o modelo

### Carregando o modelo pré-treinado (InceptionResNetV2)

### Congelando o modelo base

### Definindo o cabeçalho personalizado da rede neural

### Definindo o modelo

### Compilando o modelo

### Treinando o modelo

### Avaliação do modelo de transferência de aprendizagem

## Etapa 4: Fine tuning


Algumas questões principais:

- NEM SEMPRE é necessário realizar o Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 
- Mais informações em como usar o Fine Tuning acesse: 
 https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/

### Descongelando algumas camadas do topo do modelo base

Descomentar trecho abaixo para manter camadas congeladas. O número informado no parâmetro `fine_tuning_at` define o número das primeiras camadas que ficarão congeladas. Supondo que a rede tenha 780 camadas e seja informado o valor 500, até a camada 500 a rede ficará congelada.

### Compilando o modelo para fine tuning

### Fine tuning

### Avaliação do modelo com fine tuning

### Salvar modelo após Fine Tuning

In [14]:
model1 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model1_B7'))
model2 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model2_InceptionResnetV2'))
model3 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model3_NasNet_Large'))
model4 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model4_InceptionV3'))
model5 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model5_Xception'))
model6 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'B1-IRN50V2_3'))

In [15]:
model1._name="model1"
model2._name="model2"
model3._name="model3"
model4._name="model4"
model5._name="model5"
model6._name="model6"

In [16]:
#personalização e união
x = tf.keras.layers.Input(shape=[299, 299, 3])

model_1 = model1(x)
model_1 = tf.math.multiply(model_1, [0.0067])

model_2 = model2(x)
model_2 = tf.math.multiply(model_2, [0.2621])

model_3 = model3(x)
model_3 = tf.math.multiply(model_3, [0.0019])

model_4 = model4(x)
model_4 = tf.math.multiply(model_4, [0.0205])

model_5 = model5(x)
model_5 = tf.math.multiply(model_5, [0.0510])

model_6 = model6(x)
model_6 = tf.math.multiply(model_6, [0.6577])

base_model1 = tf.math.add(model_1, model_2)
base_model2 = tf.math.add(base_model1, model_3)
base_model3 = tf.math.add(base_model2, model_4)
base_model4 = tf.math.add(base_model3, model_5)
prediction_layer =  tf.math.add(base_model4, model_6)

#prediction_layer = tf.keras.layers.Average()([model_1, model_2, model_4, model_5])

#prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(base_model)

#final_model
model = tf.keras.models.Model(inputs = x, outputs = prediction_layer)

#optimizer
radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=251,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
optm = ranger

#compilador
model.compile(optimizer=optm, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

#model summary
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model1 (Functional)             (None, 8)            64118175    input_1[0][0]                    
__________________________________________________________________________________________________
model2 (Functional)             (None, 8)            54349032    input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 8)]          0           model1[0][0]                     
_______________________________________________________________________________________



```
model1 - Weight: 0.0067 - accuracy_score: 0.8468
model2 - Weight: 0.2621 - accuracy_score: 0.9520
model3 - Weight: 0.0019 - accuracy_score: 0.7874
model4 - Weight: 0.0205 - accuracy_score: 0.9309
model5 - Weight: 0.0510 - accuracy_score: 0.9312
model6 - Weight: 0.6577 - accuracy_score: 0.9684
DirichletEnsemble accuracy_score: 0.9741
CPU times: user 2min 22s, sys: 301 ms, total: 2min 22s
Wall time: 2min 23s
```



### Carregar modelo após Fine Tuning com os melhores pesos

### Matriz de Validação

### Execução das predições na base de testes

In [17]:
%%time

test_generator.reset()
pred=model.predict(test_generator, verbose=1)

258/258 [==============================] - 319s 1s/step
CPU times: user 6min 43s, sys: 2min 19s, total: 9min 3s
Wall time: 6min 54s


### Identificar classe predita

In [18]:
#def predClasses(pred, train_generator):
#  predicted_class_indices=np.argmax(pred,axis=1)
#
#  labels = (train_generator.class_indices)
#  labels = dict((v,k) for k,v in labels.items())
#  predictions = [labels[k] for k in predicted_class_indices]
#  return predicted_class_indices, labels, predictions

#predicted_class_indices, labels, predictions = predClasses(pred, train_generator)

In [19]:
#image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
#labels[5]
#predicted_class_indices[0]
#predictions[0]
#predicted_class_indices[0]
#pred[0].max()

In [20]:
def is_unknown(pred, th):
  unk = []
  for row in pred:
    if row.max() < th:
        #Unknown class
        unk.append(1.0)
    else:
        unk.append(0.0)
  return unk

th = 0.35

### Salvar resultado para conferência

In [21]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)
results_file = os.path.join(derm_ai_path, 'results_deepstack.csv')

def saveResultsFile(results_file,pred, test_generator, unk):
  filenames=test_generator.filenames
  filenames = [w.replace('test/', '') for w in filenames]
  filenames = [w.replace('.jpg', '') for w in filenames]
  results=pd.DataFrame({"image":filenames,
                        "MEL":pred[:,4],
                        "NV":pred[:,5],
                        "BCC":pred[:,1],
                        "AK":pred[:,0],
                        "BKL":pred[:,2],
                        "DF":pred[:,3],
                        "VASC":pred[:,7],
                        "SCC":pred[:,6],
                        "UNK": unk
                        })
  results.to_csv(results_file,index=False, float_format='%.12f')

unk = is_unknown(pred, th)
saveResultsFile(results_file, pred, test_generator, unk)

In [22]:
df_comp = pd.read_csv(results_file)
df_comp.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0034321,0.082032665610,0.594387471676,0.158217430115,0.005261190701,0.145649641752,0.004680550192,0.004500782117,0.005170242395,0.000000000000
1,ISIC_0034322,0.001062134048,0.998678088188,0.000010714355,0.000003845611,0.000135738272,0.000003125179,0.000003031219,0.000003284426,0.000000000000
2,ISIC_0034323,0.003734263591,0.001528671593,0.852527439594,0.064244262874,0.008231737651,0.001607502461,0.000157623188,0.067868582904,0.000000000000
3,ISIC_0034324,0.000241626200,0.999274015427,0.000004767862,0.000000817632,0.000275512022,0.000086862623,0.000015674117,0.000000748289,0.000000000000
4,ISIC_0034325,0.000442067074,0.999320089817,0.000003951815,0.000001325297,0.000077263940,0.000046419482,0.000007618646,0.000001222535,0.000000000000


### Rodar avaliação do modelo em relação a base de testes

### Salvar modelo com score obtido

### Carregar melhor modelo 

## Etapa 5: Tensorflow Lite

### Converter modelo para formato tflite

### Salvar arquivo convertido

### Testar modelo convertido (TFLite)
https://medium.com/towards-artificial-intelligence/testing-tensorflow-lite-image-classification-model-e9c0100d8de3

https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=6WEyRJNsR5uL

https://aihub.cloud.google.com/p/products%2F6ca92c9c-6932-45f8-8f08-15a1a6dfafc7